In [13]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [14]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [37]:
#importing raw data
jobs_original = pd.read_csv('./data/alljobs_nodupes.csv')
print(jobs_original.shape)
jobs_original.head()

(20029, 7)


,company,company_rating,industry,job_description_all_text,job_title,location,salary_data_text
0,ANZ Banking Group,4.1,data,"Consultant Data Scientist – Data Activation, M...",Data Scientist Consultant,Melbourne,NaN
1,NAB - National Australia Bank,3.8,data,Work type: Permanent Full time\nLocation: Aust...,Data Scientist,Melbourne,NaN
2,ANZ Banking Group,4.1,data,The Team\n\nThe Customer Service Operations Au...,Senior Data Scientist,Melbourne,NaN
3,NAB - National Australia Bank,3.8,data,Work type: Permanent Full time\nLocation: Aust...,Data Analytics Engineer,Melbourne,NaN
4,Coles,3.8,data,About Us\n\nThe Digital team is focused on rei...,Data Scientist,Melbourne,NaN


In [38]:
jd_text_df = jobs_original[['job_title','job_description_all_text','industry']]
jd_text_df.isna().sum()

job_title                   0
job_description_all_text    0
industry                    0
dtype: int64

In [39]:
print(jd_text_df.industry.value_counts().count())
jd_text_df.industry.value_counts()

11


retail            3182
finance           2833
manufacturing     2709
nursing           2346
hospitality       2203
data              1516
construction      1514
dataanalyst       1177
datascience       1086
earlylearning     1038
early_learning     425
Name: industry, dtype: int64

In [40]:
jd_text_df.industry = jd_text_df.industry.map(lambda x: 'data' if 'data' in x else x)
jd_text_df.industry = jd_text_df.industry.map(lambda x: 'early_learning' if 'early' in x else x)

In [41]:
print(jd_text_df.industry.value_counts().count())
jd_text_df.industry.value_counts()

8


data              3779
retail            3182
finance           2833
manufacturing     2709
nursing           2346
hospitality       2203
construction      1514
early_learning    1463
Name: industry, dtype: int64

In [17]:
tf_vect = CountVectorizer(strip_accents='unicode', 
                          stop_words='english',
                          lowercase='true',
                          token_pattern=r'\b[a-zA-Z]{3,}\b',
                          max_df=0.5,
                          min_df=10)

dtm_tf = tf_vect.fit_transform(jd_text_df.job_description_all_text)
print(dtm_tf.shape)


(20029, 11440)


In [18]:
lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=20, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [19]:
#visualizing results
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vect)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2      0.152249 -0.098126       1        1  9.605509
4     -0.084755 -0.079549       2        1  8.610974
9      0.125903  0.012086       3        1  6.710443
14     0.022296 -0.033470       4        1  6.584635
18    -0.028005  0.192959       5        1  6.547989
0      0.058733 -0.079540       6        1  5.837969
13    -0.148615 -0.051871       7        1  5.635295
19     0.092318  0.087375       8        1  5.054739
1     -0.192284 -0.052197       9        1  5.025975
15    -0.050341  0.117660      10        1  4.898028
3      0.007195 -0.051309      11        1  4.704578
11    -0.054603  0.130578      12        1  4.647903
6      0.145321 -0.040600      13        1  4.637872
12     0.095406 -0.007982      14        1  3.625202
8      0.107051 -0.145133      15        1  3.397470
17    -0.079076 -0.104792      16        1  3.379585
7     -0.247322 -0.046731      17        1  2.869439
10    -0.094634  0.046863      18        1  2.812620
5      0.114990  0.000754      19        1  2.781116
16     0.058171  0.203025      20        1  2.632659, topic_info=      Category          Freq          Term         Total  loglift  logprob
1564   Default  15123.000000          care  15123.000000  30.0000  30.0000
2651   Default  17378.000000      customer  17378.000000  29.0000  29.0000
2698   Default  16917.000000          data  16917.000000  28.0000  28.0000
1805   Default   7610.000000      children   7610.000000  27.0000  27.0000
8969   Default  10796.000000         sales  10796.000000  26.0000  26.0000
4825   Default  13101.000000        health  13101.000000  25.0000  25.0000
7799   Default   6456.000000     preferred   6456.000000  24.0000  24.0000
5888   Default   8736.000000      learning   8736.000000  23.0000  23.0000
1428   Default  27254.000000      business  27254.000000  22.0000  22.0000
8620   Default  13993.000000      required  13993.000000  21.0000  21.0000
2652   Default   9412.000000     customers   9412.000000  20.0000  20.0000
9212   Default  17183.000000       service  17183.000000  19.0000  19.0000
9056   Default   3812.000000        school   3812.000000  18.0000  18.0000
3307   Default   5191.000000         early   5191.000000  17.0000  17.0000
9831   Default   5120.000000         store   5120.000000  16.0000  16.0000
1917   Default   4714.000000      clinical   4714.000000  15.0000  15.0000
7991   Default   8366.000000       project   8366.000000  14.0000  14.0000
3367   Default   6882.000000     education   6882.000000  13.0000  13.0000
9908   Default   2918.000000      students   2918.000000  12.0000  12.0000
8726   Default   7498.000000        retail   7498.000000  11.0000  11.0000
4117   Default   8242.000000     financial   8242.000000  10.0000  10.0000
2306   Default   4465.000000  construction   4465.000000   9.0000   9.0000
1912   Default   8913.000000       clients   8913.000000   8.0000   8.0000
4115   Default   6402.000000       finance   6402.000000   7.0000   7.0000
10702  Default   4976.000000          type   4976.000000   6.0000   6.0000
7995   Default   7274.000000      projects   7274.000000   5.0000   5.0000
10235  Default   7172.000000     technical   7172.000000   4.0000   4.0000
4984   Default   6161.000000         hours   6161.000000   3.0000   3.0000
1518   Default   7222.000000     candidate   7222.000000   2.0000   2.0000
302    Default   2992.000000          aged   2992.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
6737   Topic20    171.707719         music    256.876026   3.2344  -6.3959
688    Topic20    141.983637          arts    207.021606   3.2601  -6.5860
7876   Topic20    230.996468     principal    389.564291   3.1145  -6.0993
5888   Topic20   2213.925261      learning   8736.646404   2.2644  -3.8392
11014  Topic20    345.274129     victorian    761.867507   2.8457  -5.6973
3367   Topic20  

In [42]:
lda_tf2 = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tf2.fit(dtm_tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=8, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [43]:
#visualizing results
pyLDAvis.sklearn.prepare(lda_tf2, dtm_tf, tf_vect)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.098154  0.121325       1        1  25.684826
4      0.080215 -0.099479       2        1  15.444785
6     -0.041044  0.124036       3        1  12.879285
1      0.071476 -0.165399       4        1  12.653339
7     -0.165108 -0.021690       5        1   9.770452
0      0.117714  0.028192       6        1   9.715521
3     -0.029903  0.076728       7        1   7.414907
5     -0.131504 -0.063714       8        1   6.436884, topic_info=      Category          Freq           Term         Total  loglift  logprob
1564   Default  15365.000000           care  15365.000000  30.0000  30.0000
4825   Default  13296.000000         health  13296.000000  29.0000  29.0000
1805   Default   7659.000000       children   7659.000000  28.0000  28.0000
2698   Default  16941.000000           data  16941.000000  27.0000  27.0000
5888   Default   8724.000000       learning   8724.000000  26.0000  26.0000
8969   Default  10917.000000          sales  10917.000000  25.0000  25.0000
3307   Default   5210.000000          early   5210.000000  24.0000  24.0000
1917   Default   4811.000000       clinical   4811.000000  23.0000  23.0000
2651   Default  17331.000000       customer  17331.000000  22.0000  22.0000
3367   Default   6923.000000      education   6923.000000  21.0000  21.0000
8726   Default   7611.000000         retail   7611.000000  20.0000  20.0000
9831   Default   5252.000000          store   5252.000000  19.0000  19.0000
1682   Default   5176.000000         centre   5176.000000  18.0000  18.0000
6950   Default   3522.000000        nursing   3522.000000  17.0000  17.0000
6947   Default   3255.000000          nurse   3255.000000  16.0000  16.0000
2652   Default   9360.000000      customers   9360.000000  15.0000  15.0000
7995   Default   7162.000000       projects   7162.000000  14.0000  14.0000
9217   Default  16089.000000       services  16089.000000  13.0000  13.0000
1428   Default  27327.000000       business  27327.000000  12.0000  12.0000
9056   Default   3818.000000         school   3818.000000  11.0000  11.0000
9908   Default   2921.000000       students   2921.000000  10.0000  10.0000
2306   Default   4361.000000   construction   4361.000000   9.0000   9.0000
4963   Default   2908.000000       hospital   2908.000000   8.0000   8.0000
6263   Default   3201.000000  manufacturing   3201.000000   7.0000   7.0000
7991   Default   8200.000000        project   8200.000000   6.0000   6.0000
2095   Default  11896.000000        company  11896.000000   5.0000   5.0000
6402   Default   3761.000000        medical   3761.000000   4.0000   4.0000
8635   Default   4576.000000       research   4576.000000   3.0000   3.0000
7799   Default   6425.000000      preferred   6425.000000   2.0000   2.0000
1804   Default   2190.000000      childhood   2190.000000   1.0000   1.0000
...        ...           ...            ...           ...      ...      ...
4007    Topic8   1588.247396       families   2076.950974   2.4749  -5.0654
1682    Topic8   3343.013084         centre   5176.222781   2.3059  -4.3211
337     Topic8    890.251804            aid   1210.120516   2.4362  -5.6442
3033    Topic8    938.774727        diploma   1290.605521   2.4248  -5.5912
3367    Topic8   3750.545241      education   6923.547803   2.1301  -4.2061
5078    Topic8    668.290790            iii    927.647580   2.4152  -5.9310
5888    Topic8   4260.143648       learning   8724.842431   2.0263  -4.0787
4322    Topic8    980.679028      framework   1680.007090   2.2048  -5.5475
1684    Topic8    840.963768        centres   1376.340961   2.2505  -5.7012
3368    Topic8    669.960547    educational   1044.503488   2.2990  -5.9285
1696    Topic8   1278.985889    certificate   2650.791668   2.0143  -5.2819
11415   Topic8    680.186783          young   1261.978270   2.1251  -5.9134
7974    Topic8   1710.959686        program   6289.583509   1.4413  -4.99

## Cleaning to do
???????

## Visualizations to do
1. LDA
2. Word cloud
3. Frequencies (histograms?)